# University of North Texas Campus Buildings

This notebook was prepared for the March, 2020, meeting of the Texas Association for Institutional Research, in San Antonio, Texas. It shows how institutional researchers can use a shapfile of their campus buildings to explore aggregated data on classroom enrollments. This demonstration uses data from the University of North Texas (UNT).

The *Building and Room Report* (CBM005) and the *Facilities Building Inventory* (CBM014) are reported by public colleges and universities in Texas to the Texas Higher Education Coordinating Board.

Required input files:  
  * CBM005
  * CBM014
  * Shapfiles of campus buildings - with building numbers in the metadata*
  
Required Software:
  * Python 3 - it's free and open source. If you aren't a python user, I recommend installing the newest version with [Anaconda](https://www.anaconda.com/distribution/).
  * You'll need to install the [ArcGIS API for Python](https://developers.arcgis.com/python/guide/install-and-set-up/). This python library is free for non-commercial purposes.
  * SAS (optional) - Python, R, STATA, or similar software could be used if you don't have a SAS license. 
  * GDAL/OGR which is an open source set of geospatial tools.
  * A simple text editor. I use the lightweight Microsoft Visual Studio Code.

    \*Note: The University of North Texas publish their facilities data on their [open data portal](http://data-untgis.opendata.arcgis.com/). These include building numbers.

In [1]:
#load python libraries necessary for this project.
from arcgis import GIS
import pandas as pd

pd.options.display.max_rows = 10

## Prepare data from the CBM005 and the CBM014
The next step prepares aggregates enrollment data by hour, day of the week, and building. At the end it produces a .csv file called UNT_Fall2019. This is a table with a row for each building, for each day of the week.

The SAS code is below. 

```
*This SAS program grabs enrollment by building from the CBM005 and the CBM014
to be used for mapping;

proc sql;
  create table BuildingsRaw as 
  select a.Name
      ,a.Building as BLDGCODE
      ,a.NumberOfFloors as Floors
      ,b.CourseSubject as Prefix
      ,b.CourseNumber as CourseNum
      ,b.CourseSection as Section
      ,b.Semester
      ,b.ReportYear as Year
      ,b.EnrollUGL
      ,b.EnrollUGU
      ,b.EnrollMas
      ,b.EnrollDoc
      ,b.EnrollSpec
      ,b.EnrollExcessUG
      ,b.EnrollExcessDev
      ,b.EnrollLowerExcessUG
      ,b.EnrollUpperExcessUG
      ,b.DaysOfWeek
      ,b.StartTime
      ,b.Duration
  from CBM014 a left join 
      CBM005 b on a.fice=b.fice and a.ReportYear=b.ReportYear and a.Building = b.BuildingNumber
  where a.reportYear = '2019' and b.Semester='1' and a.fice='003594';

  data Buildings (drop=EnrollUGL EnrollUGU EnrollMas EnrollDoc EnrollSpec EnrollExcessUG EnrollExcessDev EnrollLowerExcessUG EnrollUpperExcessUG);
    set BuildingsRaw;
    Enrollment = sum(EnrollUGL, EnrollUGU, EnrollMas, EnrollDoc, EnrollSpec, EnrollExcessUG, EnrollExcessDev, EnrollLowerExcessUG, EnrollUpperExcessUG);
    STime=input(StartTime,hhmmss4.);
    DurationHours=floor((Duration/60));
    DurationMinutes=(Duration*1)-(60*DurationHours);
    TimeDuration=input(cats(DurationHours,DurationMinutes),hhmmss4.);
    ETime=STime+TimeDuration;
    format STime TimeDuration ETime hhmm.;
  run;

data Buildings1;
  format D1-D7 1.;
  set Buildings;

  if index(DaysofWeek,'1') ne 0 then D1=1;
  if index(DaysofWeek,'2') ne 0 then D2=1;
  if index(DaysofWeek,'3') ne 0 then D3=1;
  if index(DaysofWeek,'4') ne 0 then D4=1;
  if index(DaysofWeek,'5') ne 0 then D5=1;
  if index(DaysofWeek,'6') ne 0 then D6=1;
  if index(DaysofWeek,'7') ne 0 then D7=1;
proc sort; by bldgcode;
run;

**********************
To do:
Create multiple rows based on BldgCode - day of the week!!!
And redo summary step!
*********************;
data Buildings2;
  set Buildings1;

 ARRAY Datarray(1:7) D1 - D7 ;
 
do i = 1 to 7;
  if Datarray(i)=1 then do;
      Day=(i);
      output;
     end;
   end;
drop D1-D7 CourseNum Prefix Section;
run;

data Buildings3;
  format HOUR0-HOUR23 6.;
  set Buildings2;
  if '00:00:00'T>=STime & '00:00:00'T<=ETime then Hour0=Enrollment;
  if '01:00:00'T>=STime & '01:00:00'T<=ETime then Hour1=Enrollment;
  if '02:00:00'T>=STime & '02:00:00'T<=ETime then Hour2=Enrollment;
  if '03:00:00'T>=STime & '03:00:00'T<=ETime then Hour3=Enrollment;
  if '04:00:00'T>=STime & '04:00:00'T<=ETime then Hour4=Enrollment;
  if '05:00:00'T>=STime & '05:00:00'T<=ETime then Hour5=Enrollment;
  if '06:00:00'T>=STime & '06:00:00'T<=ETime then Hour6=Enrollment;
  if '07:00:00'T>=STime & '07:00:00'T<=ETime then Hour7=Enrollment;
  if '08:00:00'T>=STime & '08:00:00'T<=ETime then Hour8=Enrollment;
  if '09:00:00'T>=STime & '09:00:00'T<=ETime then Hour9=Enrollment;
  if '10:00:00'T>=STime & '10:00:00'T<=ETime then Hour10=Enrollment;
  if '11:00:00'T>=STime & '11:00:00'T<=ETime then Hour11=Enrollment;
  if '12:00:00'T>=STime & '12:00:00'T<=ETime then Hour12=Enrollment;
  if '13:00:00'T>=STime & '13:00:00'T<=ETime then Hour13=Enrollment;
  if '14:00:00'T>=STime & '14:00:00'T<=ETime then Hour14=Enrollment;
  if '15:00:00'T>=STime & '15:00:00'T<=ETime then Hour15=Enrollment;
  if '16:00:00'T>=STime & '16:00:00'T<=ETime then Hour16=Enrollment;
  if '17:00:00'T>=STime & '17:00:00'T<=ETime then Hour17=Enrollment;
  if '18:00:00'T>=STime & '18:00:00'T<=ETime then Hour18=Enrollment;
  if '19:00:00'T>=STime & '19:00:00'T<=ETime then Hour19=Enrollment;
  if '20:00:00'T>=STime & '20:00:00'T<=ETime then Hour20=Enrollment;
  if '21:00:00'T>=STime & '21:00:00'T<=ETime then Hour21=Enrollment;
  if '22:00:00'T>=STime & '22:00:00'T<=ETime then Hour22=Enrollment;
  if '23:00:00'T>=STime & '23:00:00'T<=ETime then Hour23=Enrollment;
run;

proc sort data=Buildings3; by BLDGCODE Day Hour0-Hour23; run;

proc summary data=buildings3 nway;
  class BLDGCODE Day;
  var Hour0-Hour23;
  id Name Floors;
  output out=BldgSummary (drop=_type_ _freq_) sum=;
run;

data MaxEnr1;
  set BldgSummary;
  format MaxEnr 6.;
maxEnr=max(of Hour0-Hour23);
proc sort; by BldgCode maxEnr;run;
data MaxEnr2 (keep=bldgcode MaxEnr);
  set MaxEnr1;
  by BldgCode;
  if last.BldgCode;
run;

data final;
  merge BldgSummary (in=A)
        MaxEnr2 (in=B);
  by BldgCode;

*Mask values under 5;
ARRAY Datarray(24) Hour0 - Hour23 ;
do i = 1 to 24;
  if Datarray(i)<5 then Datarray(i)=0;
end;
drop i;
if MaxEnr>=5;

Height=Floors*6; *The floors probably AREN'T 6 meters high! but it looks more interesting on a map;
run;

proc export data=final
   outfile="UNT_Facilities\UNT_Fall2019.csv" 
   dbms=csv replace;
run;
```

### Let's look at the .csv of enrollment totals by hour and building.

In [2]:
#Read csv as pandas dataframe
df = pd.read_csv("UNT_Fall2019.csv", dtype={'BLDGCODE': 'str'})

#Make layer for monday and view the output
Monday = df[df.Day==1]
print(Monday)

    BLDGCODE  Day                     NAME  Floors  HOUR0  HOUR1  HOUR2  \
0       0102    1  AUDITORIUM-ENGLISH BLDG       4      0      0      0   
5       0104    1                SAGE HALL       4      0      0      0   
10      0106    1            MATTHEWS HALL       4      0      0      0   
16      0107    1               CURRY HALL       3      0      0      0   
21      0108    1    GENERAL ACADEMIC BLDG       5      0      0      0   
..       ...  ...                      ...     ...    ...    ...    ...   
161     0173    1             RAWLINS HALL       6      0      0      0   
166     0184    1           GOOLSBY CHAPEL       1      0      0      0   
171     0190    1  DISCOVERY PARK BUILDING       3      0      0      0   
179     0306    1    PERFORMING ARTS ANNEX       1      0      0      0   
182     5000    1               UNT FRISCO       1      0      0      0   

     HOUR3  HOUR4  HOUR5  ...  HOUR16  HOUR17  HOUR18  HOUR19  HOUR20  HOUR21  \
0        0      0 

## Download the facilities shapefile from the [University of North Texas Facilities GIS](http://data-untgis.opendata.arcgis.com/) open data portal

The file you need is called [Building_Cartographic_Shape](https://data-untgis.opendata.arcgis.com/datasets/building-cartographic-shape). Their open data portal provides information about the dataset. If you select the *Data* tab, you can look at the data contained in the attribute table. Shapefiles usually come zipped and are actually comprised of about half a dozen files.

Unzip the contents into the folder *UNT Shapfile/*. 


### Load the shapefile of UNT buildings

In the next step, we load the file and look at the variables it comes with. Notice that one of those variables is **'BLDGCODE'**. That's the one we'll need.

In [3]:
# get UNT shapefile
UNT_raw = pd.DataFrame.spatial.from_featureclass("UNT Shapefile/Building_Cartographic_Shape.shp")

# get a list of columns
list(UNT_raw)

['FID',
 'FID_1',
 'Northing',
 'Easting',
 'LON',
 'LAT',
 'SITECODE',
 'OWNEDBY',
 'MANAGEDBY',
 'IMG',
 'IMGFILE',
 'NOTES',
 'BLDGCODE',
 'NAME',
 'CATEGORY',
 'THECB_TYPE',
 'LEEDLEVEL',
 'GF_BLDG',
 'GF_RMLIST',
 'IMGSVR',
 'RefName',
 'HISTORY',
 'SHAPE_STAr',
 'SHAPE_STLe',
 'Shape__Are',
 'Shape__Len',
 'SHAPE']

In [4]:
# Keep just the columns you need
UNT=UNT_raw[['BLDGCODE', 'SHAPE']]

# view a few records to confirm we only have the Building Code and shapedata
UNT.head()

,BLDGCODE,SHAPE
0,0313,"{'rings': [[[-97.1516735080242, 33.20626862726..."
1,-99,"{'rings': [[[-97.1512143289545, 33.20722744197..."
2,-99,"{'rings': [[[-97.1514594687618, 33.20722942032..."
3,0171,"{'rings': [[[-97.1521253511132, 33.20733396099..."
4,0122,"{'rings': [[[-97.1526414442144, 33.20958428825..."


In [5]:
#Make layer for each day of the week
Monday = df[df.Day==1]
Tuesday = df[df.Day==2]
Wednesday = df[df.Day==3]
Thursday = df[df.Day==4]
Friday = df[df.Day==5]
Saturday = df[df.Day==6]

#print monday to look at the data
print(Monday)

#merge hourly enrollment data for each day of the week, with the building polygons
right = UNT
MondayPolys = pd.merge(Monday, right, how='left', on=['BLDGCODE'] , validate="one_to_many")
TuesdayPolys = pd.merge(Tuesday, right, how='left', on=['BLDGCODE'] , validate="one_to_many")
WednesdayPolys = pd.merge(Wednesday, right, how='left', on=['BLDGCODE'] , validate="one_to_many")
ThursdayPolys = pd.merge(Thursday, right, how='left', on=['BLDGCODE'] , validate="one_to_many")
FridayPolys = pd.merge(Friday, right, how='left', on=['BLDGCODE'] , validate="one_to_many")
SaturdayPolys = pd.merge(Saturday, right, how='left', on=['BLDGCODE'] , validate="one_to_many")


#export the Building polygons, merged with enrollment totals, to shapefiles
MondayPolys.spatial.to_featureclass(location=r"Data/MondayPolys.shp")
TuesdayPolys.spatial.to_featureclass(location=r"Data/TuesdayPolys.shp")
WednesdayPolys.spatial.to_featureclass(location=r"Data/WednesdayPolys.shp")
ThursdayPolys.spatial.to_featureclass(location=r"Data/ThursdayPolys.shp")
FridayPolys.spatial.to_featureclass(location=r"Data/FridayPolys.shp")
SaturdayPolys.spatial.to_featureclass(location=r"Data/SaturdayPolys.shp")




    BLDGCODE  Day                     NAME  Floors  HOUR0  HOUR1  HOUR2  \
0       0102    1  AUDITORIUM-ENGLISH BLDG       4      0      0      0   
5       0104    1                SAGE HALL       4      0      0      0   
10      0106    1            MATTHEWS HALL       4      0      0      0   
16      0107    1               CURRY HALL       3      0      0      0   
21      0108    1    GENERAL ACADEMIC BLDG       5      0      0      0   
..       ...  ...                      ...     ...    ...    ...    ...   
161     0173    1             RAWLINS HALL       6      0      0      0   
166     0184    1           GOOLSBY CHAPEL       1      0      0      0   
171     0190    1  DISCOVERY PARK BUILDING       3      0      0      0   
179     0306    1    PERFORMING ARTS ANNEX       1      0      0      0   
182     5000    1               UNT FRISCO       1      0      0      0   

     HOUR3  HOUR4  HOUR5  ...  HOUR16  HOUR17  HOUR18  HOUR19  HOUR20  HOUR21  \
0        0      0 

'C:\\Users\\John\\Desktop\\UNTBuildingData2.8.2020\\GitRepository\\Data\\SaturdayPolys.shp'

# From here go to linux (bash) to convert to geojson

### From the GDAL/OGR library, use the ogr2ogr tool to convert the shapefiles into geojson

ogr2ogr -f GeoJSON geojson/UNT_Monday.json Data/MondayPolys.shp -progress

ogr2ogr -f GeoJSON geojson/UNT_Tuesday.json Data/TuesdayPolys.shp -progress

ogr2ogr -f GeoJSON geojson/UNT_Wednesday.json Data/WednesdayPolys.shp -progress

ogr2ogr -f GeoJSON geojson/UNT_Thursday.json Data/ThursdayPolys.shp -progress

ogr2ogr -f GeoJSON geojson/UNT_Friday.json Data/FridayPolys.shp -progress

ogr2ogr -f GeoJSON geojson/UNT_Saturday.json Data/SaturdayPolys.shp -progress



### Assign the geojson data to variables. Save the files as .js

I copied each .json file, changed file type, and added one line at the top as follows:
  * I saved UNT_Monday.json as UNT_Monday.js
  * At the very top of the file I added `const monGeojson = `. This assigned the geojson data to a variable (constant) called *monGeojson*.
  * Do the same for each day of the week